<a href="https://colab.research.google.com/github/alienlifeform/exotic-colab/blob/main/refactor1_EXOTIC_in_Action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EXOTIC: See it in Action**

Welcome! This is a step-by-step way to get started, using Google's Collaboratory ("Colab") to run EXOTIC's Python code in your browser on a set of sample telescope images. The steps are:

* **SETUP** 
 * Import the EXOTIC code libraries
* **LOAD TELESCOPE IMAGES** 
 * Download sample images taken for a previous transition of planet Hat-p-32b
* **IDENTIFY TARGET STAR AND COMPARISON STARS** 
 * Find the target star (Hat-p-32) and several comparison stars
 * Enter their coordinates to calibrate the baseline brightness
* **RUN EXOTIC** 
 * Watch EXOTIC compare your telescope images to the AAVSO starchart to see if the brightness of your target star changes, signifying a transition. [(What is a transition?)](https://exoplanets-5.client.mooreboeck.com/alien-worlds/ways-to-find-a-planet/#/2)

*INSTRUCTIONS:* Simply click the Screen Shot 2022-09-08 at 10.26.08 AM.png, Screen Shot 2022-09-08 at 10.19.48 AM.png or Screen Shot 2022-09-08 at 10.19.58 AM.png icon to the left of each code snippet, it will run the code for each step. Wait for it to finish before moving to the next step, and watch for the output describing what is happening.

*For this tutorial, it is recommended that you do not click Screen Shot 2022-09-08 at 10.47.22 AM.png*

# **Step 1: Import the EXOTIC code libraries**

---
> Estimated time: 1 minute
---

*ATTENTION:* A popup will appear titled "**Warning: This notebook was not authored by Google.**"

Read it, then click "**Run Anyway**"... we stand behind our code!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title <font face="Helvetica" color='#702020'>&lt;- Click to import the EXOTIC code libraries</font> { vertical-output: true }

##############################################################
%%capture step_capture --no-display
print("Starting application...")

from IPython.display import display, HTML, Javascript
import random

# Grabs a stylesheet, call this before any html output in cell
def importCustomStyles():
  #custom_stylesheet_url = 'https://exoplanets.nasa.gov/system/exotic/colab.css?i=' + str(random.random())
  exotic_css = open("/content/drive/MyDrive/exotic.css")
  display(HTML('<style>' + exotic_css.read() + '</style>'))
  #display(HTML('Getting stylesheet from ' + custom_stylesheet_url))

def setupDisplay():
  # importCustomStyles()
  exotic_html = open("/content/drive/MyDrive/exotic.html")
  display(HTML(exotic_html.read()))


def displayStep(message):
  js_code = '''\
                var container = document.querySelector("#output-body ul.step_container");\
                container.innerHTML += '<li class="step1">{m}</li>';
                '''.format(m=message)
  display(Javascript(js_code))

def makeContainer(container_class):
  display(HTML('<div class="' + container_class + '"></div>'))

def appendToContainer(container_class, html_chunk):
  js_code = '''\
                var container = document.querySelector("#output-body .{c}");\
                container.innerHTML += '{html_chunk}';
                '''.format(c=container_class, m=html_chunk)
  display(Javascript(js_code))
  
def expandableSection(content):
  html_content = '''\
    <div class="expandable" onclick="show_or_hide(this)">
      <div class="expand_text">+ MORE</div>
      <div class="hidden">
        {expand_content}
      </div>
    </div>
    '''.format(expand_content=content)
  display(HTML(html_content))


display(HTML('<p class="bookend">START: Importing libraries</p>'))

setupDisplay()

display(HTML('<ul class="step_container"></ul>'))
displayStep('Time, ProgressBar')

import time
import progressbar

# Creates a progress bar that just runs for `seconds` number of seconds
def showProgress(seconds):
  with progressbar.ProgressBar(max_value=100) as bar:
    for idx, val in enumerate(range(100)):
      time.sleep(seconds/100)
      bar.update(idx)

# Avoids scroll-in-the-scroll in the entire Notebook
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))

get_ipython().events.register('pre_run_cell', resize_colab_cell)
displayStep('IPython.display')

displayStep('Bokeh.io')
import bokeh.io

displayStep('EXOTIC <span class="comment">(This will take up to a minute, please wait... and ignore any warning that may ask you to "RESTART RUNTIME")</span>')
!pip install exotic --upgrade
display(HTML('<br /><br /><p class="step"><span class="comment">Reminder, if there is a "RESTART RUNTIME" warning button above, ignore it!</span></p>'))

# from exotic.api.plotting import plot_image
displayStep('NASAExoplanetArchive')
from exotic.exotic import NASAExoplanetArchive, get_wcs, find_target

displayStep('Astropy')
from astropy.time import Time

displayStep('Utils')
from barycorrpy import utc_tdb
import numpy as np
from io import BytesIO
from astropy.io import fits
from scipy.ndimage import label
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Viridis256
from bokeh.models import ColorBar, LinearColorMapper, LogColorMapper, LogTicker
from bokeh.models import BoxZoomTool,WheelZoomTool,ResetTool,HoverTool,PanTool,FreehandDrawTool
from bokeh.io import output_notebook
from pprint import pprint
from IPython.display import Image
from ipywidgets import widgets, HBox
from skimage.transform import rescale, resize, downscale_local_mean
import copy

displayStep('Matlab')
import matplotlib.pyplot as plt

displayStep('Scipy')
from scipy.stats import gaussian_kde

displayStep('OS/RE/JSON')
import os
import re
import json
import subprocess

displayStep('Google Utils')
from google.colab import drive, files

expandableSection('<p>This is some expandable stuff</p>')
display(HTML('<p class="bookend">DONE: Importing libraries. <b>You may move on to the next step.</b></p>'))

# **Step 2: Load Telescope Images**

---
> Estimated time: 10 seconds
---

For this tutorial, we are using a known transit, that of Hat-p-32b around the star Hat-p-32, taken on December 20, 2017. 

Next time, you will do the same steps with images from your own telescope images taken during the expected transit of a likely exoplanet.

In [ ]:
#@title <font face="Helvetica" class="button" color='#702020'>&lt;- Click to load telescope images</font>
#%%capture step_capture --no-display

display(HTML('<p class="bookend">START: Loading sample data</p>'))
setupDisplay()
display(HTML('<ul class="step_container"></ul>'))
displayStep('Ensuring sample data is loaded...')


#
# Delete existing sample data by changing to `rebuild = "true"`
#
rebuild = "true"  
if rebuild == "true":
  if os.path.isdir("/content/EXOTIC/exotic-in-action"):
    %rm -rf /content/EXOTIC/exotic-in-action
    print("'Rebuild' flag is 'true'... Removing old data at /content/EXOTIC/exotic-in-action")

#
# Download sample files if necessary
#
#sample_data_source = "https://github.com/rzellem/EXOTIC_sampledata.git" # goes into sample_data_target_folder
sample_data_source = "https://github.com/alienlifeform/exotic.git" # goes into sample_data_target_parent
sample_data_target_parent = "/content/EXOTIC/exotic-in-action"
sample_data_target_folder = "/content/EXOTIC/exotic-in-action/sample-data"
sample_data_target_child = "/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017"
if os.path.isdir(sample_data_target_child):
  displayStep('Skipping... Sample data already loaded')

else:
  displayStep('Downloading data from ' + sample_data_source)
  #git_rv = !git clone {sample_data_source} {sample_data_target_folder}
  git_rv = !git clone {sample_data_source} {sample_data_target_parent}
  displayStep('Sample data successfully loaded for Hat-p-32 b')

#
# Show files found
#
#!du -hd0 --exclude ".*" /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017
numfiles = !ls {sample_data_target_child} | grep -ci FITS

print('You have ' + str(numfiles[0]) + ' telescope image (.FITS) files</li>')

display(HTML('<p class="bookend">DONE: Loading sample data. <b>You may move on to the next step.</b></p>'))

# to mount the user's Google Drive... the original way
#drive.mount('/content/drive', force_remount=True)


# **Step 3:  Download planetary data from the Exoplanets Archive**

---
> Estimated time: 10 seconds
---

This temporary step allows us to accurately identify the planet for the next step. Rob will help Bergen remove this step in the future.


In [ ]:
#@title <font face="Helvetica" class="button" color='#702020'>&lt;- Click to download Planetary data for Hat-p-32 b</font> { vertical-output: true }

##############################################################

def fix_planetary_params (p_param_dict):
  for param in p_param_dict.keys():
    if param == "Target Star RA" or param == "Target Star Dec" or param == "Planet Name" or param == "Host Star Name" or param == "Argument of Periastron (deg)":
      continue
    val = p_param_dict[param]
    if val == 0.0 or np.isnan(float(val)):
      if param == "Orbital Eccentricity (0 if null)":
        continue
      if param == "Ratio of Planet to Stellar Radius (Rp/Rs)":
        p_param_dict[param] = 0.151
      if param == "Ratio of Planet to Stellar Radius (Rp/Rs) Uncertainty":
        p_param_dict[param] = 0.151
        if p_param_dict["Host Star Name"] == "Qatar-6":
          p_param_dict[param] = 0.01
      print(f"\nIn the planetary parameters from the NASA Exoplanet Archive, \n\"{param}\" is listed as {val}.\n\n**** This might make EXOTIC crash. ****\n\nIf the parameter is *not* changed below, please edit it\nin the inits file before running EXOTIC.\n")
  p_param_string = json.dumps(p_param_dict)

  planetary_params = "\"planetary_parameters\": {\n"
  num_done, num_total = 0, len(p_param_dict.keys())
  for key, value in p_param_dict.items():
    num_done += 1
    if key == "Target Star RA" or key == "Target Star Dec" or key == "Planet Name" or key == "Host Star Name":
      planetary_params = planetary_params + str(f"    \"{key}\": \"{value}\",\n")
    else:
      if num_done < num_total:
        planetary_params = planetary_params + str(f"    \"{key}\": {value},\n")
      else:
        planetary_params = planetary_params + str(f"    \"{key}\": {value}\n")
  planetary_params = planetary_params + "}"

  return(planetary_params)

##############################################################

display(HTML('<p class="bookend">START: Download planetary parameters</p>'))
setupDisplay()
display(HTML('<ul class="step_container"></ul>'))
displayStep('Connecting with NASA Exoplanet Archive')

#target=input("Please enter the name of your exoplanet target: ")
target="Hat-p-32 b"
targ = NASAExoplanetArchive(planet=target)
target = targ.planet_info()[0]

displayStep('Found target ' + target + ' in the NASA Exoplanet Archive')

planetary_params = ""
if not targ.resolve_name():
  print("Sorry, can't find your target in the Exoplanet Archive.  Unfortunately, this")
  print("isn't going to work until I can find it.  Please re-run this cell, trying")
  print("different formats for your target name, until the target is located.")
  print("Looking it up in the NASA Exoplanet Archive at https://exoplanetarchive.ipac.caltech.edu/")
  print("might help you know where to put the spaces and hyphens and such.")
else:
  p_param_string = targ.planet_info(fancy=True)
  planetary_params = "\"planetary_parameters\": "+p_param_string
  p_param_dict = json.loads(p_param_string)
  planetary_params = fix_planetary_params(p_param_dict)
  displayStep('Loading NASA Exoplanet Archive planetary parameters for ' + target)
  print(planetary_params)

  displayStep('Planetary parameters saved to memory')

display(HTML('<p class="bookend">DONE: Download planetary parameters. <b>You may move on to the next step.</b></p>'))


# **Step 4: Identify target star and comparison stars**

---
> Estimated time: 4 minutes
---

In this multi-part step, you will use your stargazing skills to identify *where in the sky* the target star (Hat-p-32) is. For you to do:

1. **Watch the Video:** How to determine a star's coordinates (2 minutes)
1. **Try it yourself:** Enter the coordinates for the target and comparison stars (2 minutes)

In [ ]:
#@title <font face="Helvetica" class="button" color='#702020'>&lt;- Click to watch the video: "How to enter a star's coordinates"</font>

setupDisplay()

display(HTML('<h3>VIDEO: How to enter a star\'s coordinates</h3>'));
display(HTML('<p>Video Length: 95 seconds (Segment starts at 1:37, and you can stop at 3:12)</p>'))
display(HTML('<p>Please note this video shows an older interface to EXOTIC, but the principles remain the same. Click the image below to begin.</p>'));

#display(HTML('<iframe width="840" height="472" src="https://drive.google.com/file/d/10zlQRgT8iV3dSe0FVW7tiL-V86ewai_1/preview?t=1m37s" allow="autoplay" allowfullscreen></iframe>'))
display(HTML('<video src="https://exoplanets.nasa.gov/system/exotic/exotic-identify-stars.mov"  controls autoplay />'))


In [ ]:
#@title <font face="Helvetica" class="button" color='#702020'>&lt;- Click to identify target and comparison stars yourself</font>


# If the user presses enter to run the sample data, download sample data if needed and
# put it into a sample-data directory at the top level of the user's Gdrive.  Count
# the .fits files (images) and .json files (inits files) in the directory entered 
# by the user (or in the sample-data directory if the user pressed enter).  If 
# there are at least 20 .fits files, assume this is a directory of images and display
# the first one in the series.  If there is exactly one inits file in the directory, 
# show the specified target and comp coords so that the user can check these against
# the displayed image.  Otherwise, prompt for target / comp coords and make an inits 
# file based  on those (save this new inits file in the folder with the output files 
# so that the student can consult it later).  Finally, run EXOTIC with the newly-made 
# or pre-existing inits file, plus any other inits files in the directory.

#########################################################

def display_image(filename):
    #print(f"{filename}")
    hdu = fits.open(filename)

# replace 0's with extension      # Stuff to put in if the hdul ever becomes a problem.    
#    extension = 0
#    image_header = hdul[extension].header
#    while image_header["NAXIS"] == 0:
#      extension += 1
#      image_header = hdul[extension].header

    dheader = dict(hdu[0].header)
  
    data = hdu[0].data
    megapixel_factor = (data.shape[0])*(data.shape[1])/1000000.0
    if megapixel_factor > 5:
      print(f"Downsampling image because it has {megapixel_factor} megapixels.")
      image_downscaled = downscale_local_mean(data, (2, 2)).astype(int)
      data = image_downscaled
    
    max_y = len(data)
    max_x = len(data[0])
    p_height = 500
    p_width = int((p_height/max_y) * max_x)

    # quick hot pixel/ cosmic ray mask
    # mask, cdata = detect_cosmics(
    #     data, psfmodel='gauss',
    #     psffwhm=4, psfsize=2*round(4)+1, # just a guess
    #     sepmed=False, sigclip = 4.25,
    #     niter=3, objlim=10, cleantype='idw', verbose=False
    # )

    # create a figure with text on mouse hover
    fig = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], plot_width=p_width, plot_height=p_height,
        tools=[PanTool(),BoxZoomTool(),WheelZoomTool(),ResetTool(),HoverTool()])
    fig.x_range.range_padding = fig.y_range.range_padding = 0

    r = fig.multi_line('x', 'y', source={'x':[],'y':[]},color='white',line_width=3)
    fig.add_tools(FreehandDrawTool(renderers=[r]))

    # set up a colobar + data range
    color_mapper = LogColorMapper(palette="Cividis256", low=np.percentile(data, 55), high=np.percentile(data, 99))

    # must give a vector of image data for image parameter
    fig.image(
        image=[data],
          x=0, y=0, dw=hdu[0].data.shape[1], dh=hdu[0].data.shape[0],
          level="image", color_mapper=color_mapper
    )
    fig.grid.grid_line_width = 0.5

    color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                         label_standoff=12, border_line_color=None, location=(0,0))

    fig.add_layout(color_bar, 'right')

    show(fig)

#########################################################

def floats_to_ints(l):
  while (True):
#    print (l)
    m = re.search(r"^(.*?)(\d+\.\d+)(.*?)$", l)
    if m:
      start, fl, end = m.group(1), float(m.group(2)), m.group(3)
      l = start+str("%.0f" % fl)+end
    else:
      return(l)
  
#########################################################

# Find a field in the image fits header or prompt the user to enter the corresponding
# value.

def check_dir(p):
  p = p.replace("\\", "/")

  if not(os.path.isdir(p)):
    print(f"Problem: the directory {p} doesn't seem to exist")
    print("on your Gdrive filesystem.")
    return("")
  return(p)

#########################################################

def add_sign(var):
  str_var = str(var)
  m=re.search(r"^[\+\-]", str_var)
  if m:
    return(str_var)
  if float(var) >= 0:
    return(str("+%.6f" % float(var)))
  else:
    return(str("-%.6f" % float(var)))

#########################################################

def get_val(hdr, ks):
  for key in ks:
    if key in hdr.keys():
      return hdr[key]
    if key.lower() in hdr.keys():
      return hdr[key.lower()]
    new_key = key[0]+key[1:len(key)].lower()  # first letter capitalized
    if new_key in hdr.keys():
      return hdr[new_key]
  return("")

#########################################################

def process_lat_long(val, key):
  m = re.search(r"\'?([+-]?\d+)[\s\:](\d+)[\s\:](\d+\.?\d*)", val)
  if m:
    deg, min, sec = float(m.group(1)), float(m.group(2)), float(m.group(3))
    if deg < 0:
      v = deg - (((60*min) + sec)/3600)
    else:
      v = deg + (((60*min) + sec)/3600)
    return(add_sign(v))
  m = re.search("^\'?([+-]?\d+\.\d+)", val)
  if m:
    v = float(m.group(1))
    return(add_sign(v))
  else:
    print(f"Cannot match value {val}, which is meant to be {key}.")

#########################################################

# Convert a MicroObservatory timestamp (which is in local time) to UTC.

def convert_Mobs_to_utc(datestamp, latitude, longitude, height):

#  print(datestamp)
  t = Time(datestamp[0:21], format='isot', scale='utc')
  t -= 0.33

  return(str(t)[0:10])

#########################################################

def find (hdr, ks, obs):
  # Special stuff for MObs and Boyce-Astro Observatories
  boyce = {"FILTER": "ip", "LATITUDE": "+32.6135", "LONGITUD": "-116.3334", "HEIGHT": 1405 }
  mobs = {"FILTER": "V", "LATITUDE": "+37.04", "LONGITUD": "-110.73", "HEIGHT": 2606 }

  if "OBSERVAT" in hdr.keys() and hdr["OBSERVAT"] == 'Whipple Observatory':
    obs = "MObs"

#  if "USERID" in hdr.keys() and hdr["USERID"] == 'PatBoyce':
#    obs = "Boyce"

  if obs == "Boyce":
    boyce_val = get_val(boyce, ks)
    if (boyce_val != ""):
      return(boyce_val)
  if obs == "MObs":
    mobs_val = get_val(mobs, ks)
    if (mobs_val != ""):
      return(mobs_val)

  val = get_val(hdr, ks)

  if ks[0] == "LATITUDE" and val != "":         # Because EXOTIC needs these with signs shown.
    return(process_lat_long(str(val), "latitude"))
  if ks[0] == "LONGITUD" and val != "":
    return(process_lat_long(str(val), "longitude"))

  if (val != ""):
    return(val)

  print(f"\nI cannot find a field with any of these names in your image header: \n{ks}.")
  print("Please enter the value (not the name of the header field, the actual value) that should")
  print("be used for the value associated with this field.\n")
  if ks[0] == "HEIGHT":
    print("The units of elevation are meters.")
  
  value = input("")

  return(value)

###############################################

def look_for_calibration(image_dir):
  darks_dir, flats_dir, biases_dir = "null", "null", "null"

  m = re.search(r"(.*?)(\d\d\d\d\-\d\d\-\d\d)\/images", image_dir)  # This handles the way I set up the MObs image paths for my seminar teams.
  if m:
    prefix, date = m.group(1), m.group(2)
    darks = prefix+date+"/darks"
    if os.path.isdir(darks):
      darks_dir = str("\""+darks+"\"")
      
  d_names = ["dark", "darks", "DARK", "DARKS", "Dark", "Darks"]  # Possible names for calibration image directories.
  f_names = ["flat", "flats", "FLAT", "FLATS", "Flat", "Flats"]
  b_names = ["bias", "biases", "BIAS", "BIASES", "Bias", "Biases"]

  for d in d_names:
    if os.path.isdir(os.path.join(image_dir, d)):
      darks_dir = str("\""+os.path.join(image_dir, d)+"\"")
      break

  for f in f_names:
    if os.path.isdir(os.path.join(image_dir, f)):
      flats_dir = str("\""+os.path.join(image_dir, f)+"\"")
      break

  for b in b_names:
    if os.path.isdir(os.path.join(image_dir, b)):
      biases_dir = str("\""+os.path.join(image_dir, b)+"\"")
      break

  return(darks_dir, flats_dir, biases_dir)

###############################################

# Writes a new inits file into the directory with the output plots.  This prompts
# for needed information that it cannot find in the fits header of the first image.

def make_inits_file(planetary_params, image_dir, output_dir, first_image, targ_coords, comp_coords, obs, aavso_obs_code, sample_data):
  inits_file_path = output_dir+"inits.json"
  hdul = fits.open(first_image)
  hdr = dict(hdul[0].header)

  min, max = "null", "null"
  filter = find(hdr, ['FILTER', 'FILT'], obs)
  if filter == "w":
    filter = "PanSTARRS-w"
    min = "404"
    max = "846"
  if filter == "Clear":
    filter = "V"
  if filter == "ip":
    min = "690"
    max = "819"
  if filter == "EXO":
    filter = "CBB"
  if re.search(r"Green", filter, re.IGNORECASE):
    filter = "SG"
    
  date_obs = find(hdr,["DATE", "DATE_OBS", "DATE-OBS"], obs)
  date_obs = date_obs.replace("/", "_")
  longitude = find(hdr,['LONGITUD', 'LONG', 'LONGITUDE', 'SITELONG'],obs)
  latitude = find(hdr,['LATITUDE', 'LAT', 'SITELAT'],obs)
  height = int(find(hdr, ['HEIGHT', 'ELEVATION', 'ELE', 'EL', 'OBSGEO-H', 'ALT-OBS', 'SITEELEV'], obs))
  obs_notes = "N/A"
  sec_obs_code = "N/A"

  mobs_data = False
  # For MObs, the date is local rather than UTC, so convert.
  if "OBSERVAT" in hdr.keys() and hdr["OBSERVAT"] == 'Whipple Observatory':
    date_obs = convert_Mobs_to_utc(date_obs, latitude, longitude, height)
    weather = hdr["WEATHER"] 
    temps = float(hdr["TELTEMP"]) - float(hdr["CAMTEMP"])
    obs_notes = str("First image seeing %s (0: poor, 100: excellent), Teltemp - Camtemp %.1f.  These observations were conducted with MicroObservatory, a robotic telescope network managed by the Harvard-Smithsonian Center for Astrophysics on behalf of NASA's Universe of Learning. This work is supported by NASA under award number NNX16AC65A to the Space Telescope Science Institute." % (weather, temps))
    sec_obs_code = "MOBS"  
    mobs_data = True
  
  if aavso_obs_code == "":
      aavso_obs_code = "N/A"

  obs_date = date_obs[0:10]
  (darks_dir, flats_dir, biases_dir) = look_for_calibration(image_dir)

  with open(inits_file_path, 'w') as inits_file:
    inits_file.write("""
{
  %s,
    "user_info": {
            "Directory with FITS files": "%s",
            "Directory to Save Plots": "%s",
            "Directory of Flats": %s,
            "Directory of Darks": %s,
            "Directory of Biases": %s,

            "AAVSO Observer Code (N/A if none)": "%s",
            "Secondary Observer Codes (N/A if none)": "%s",

            "Observation date": "%s",
            "Obs. Latitude": "%s",
            "Obs. Longitude": "%s",
            "Obs. Elevation (meters)": %d,
            "Camera Type (CCD or DSLR)": "CCD",
            "Pixel Binning": "1x1",
            "Filter Name (aavso.org/filters)": "%s",
            "Observing Notes": "%s",

            "Plate Solution? (y/n)": "y",
            "Align Images? (y/n)": "y",

            "Target Star X & Y Pixel": %s,
            "Comparison Star(s) X & Y Pixel": %s
    },    
    "optional_info": {
            "Pixel Scale (Ex: 5.21 arcsecs/pixel)": null,
            "Filter Minimum Wavelength (nm)": %s,
            "Filter Maximum Wavelength (nm)": %s
    }
}
""" % (planetary_params, image_dir, output_dir, flats_dir, darks_dir, biases_dir, 
       aavso_obs_code, sec_obs_code, obs_date, latitude, longitude, height, filter, 
       obs_notes, targ_coords, comp_coords, min, max))


  # print("\nWithin your folder of images, there is now a new folder called EXOTIC_output.")
  # print("This folder contains an initialization file for EXOTIC called inits.json.")
  # print("The same folder will contain the output files and images when EXOTIC finishes running.")

  if not mobs_data:  
    print(f"\nThe inits.json file currently says that your observatory latitude was {latitude} deg,")
    print(f"longitude was {longitude} deg, and elevation was {height}m.  \n")
    print("*** If any of these are incorrect, please change them in the inits.json file. ***")
    print("*** (Please make sure that Western longitudes have a negative sign! ***")
    print("*** TheSkyX sometimes stamps Western longitudes as positive; this needs to be switched! ***\n")

  #print("\nNOTE: At this point in EXOTIC, you would have the opportunity change parameters in the inits file.")

  #print("\nIf you want to change anything in the inits file, please do that now.")
  #print("When you are done, press enter to continue.")
  #okay = input()

  time.sleep(1)
  #print("Your coordinates have been saved in the initialization file here: " + inits_file_path)
  
  return(inits_file_path)

#########################################################



display(HTML('<p class="bookend">START: Compare telescope image and star chart</p>'))
setupDisplay()
display(HTML('<ul class="step_container"></ul>'))
bokeh.io.output_notebook()
sample_data = False

p = "sample-data/HatP32Dec202017"
p = check_dir(os.path.join("/content/EXOTIC/exotic-in-action/", p))
output_dir = os.path.join(p, "EXOTIC_output/")      
                                         
inits = []    # array of paths to any inits files found in the directory
all_files = [f for f in sorted(os.listdir(p)) if os.path.isfile(os.path.join(p, f))]
fits_count, first_image = 0, ""
for f in all_files:
  if re.search(r"\.f[itz]+s?\.?g?z?$", f, re.IGNORECASE):
    if first_image == "":
      first_image = os.path.join(p, f)
    fits_count += 1
  if re.search(r"\.json$", f, re.IGNORECASE):
    inits.append(os.path.join(p, f))

#print('Found ' + str(fits_count) + ' image files and ' + str(len(inits)) + ' initialization files in the directory')

#if fits_count >= 19:                  # more than 20 images in the folder --> run EXOTIC on these images
if fits_count >= 2:                  # more than 3 images in the folder --> run EXOTIC on these images -bm
  if len(inits) == 1:                 # one inits file exists
    inits_path = os.path.join(p, inits[0])
    with open(inits_path) as i_file:
      inits_data = i_file.read()
      d = json.loads(inits_data)
      targ_coords = d["user_info"]["Target Star X & Y Pixel"]
      comp_coords = d["user_info"]["Comparison Star(s) X & Y Pixel"]
      input_dir = d["user_info"]["Directory with FITS files"]
      if input_dir != p:
        print(f"The directory with fits files should be {p} but your inits file says {input_dir}.")
        print("This may or may not cause problems.  Just letting you know.")
      print(f"Coordinates from your inits file:\ntarget: {targ_coords}\ncomps: {comp_coords}")
      output_dir = d["user_info"]["Directory to Save Plots"]
      
  else:                               # no inits file: display image and prompt for target, comp coords;
                                      # then make an inits file and put it into the output directory (with the plots)
    #print("There are either 0 or > 1 inits files in your image directory, so we'll make a new one.")
    #print("Displaying first image:")

    display(HTML('<br clear="all"/>'))
    display(HTML('<hr />'))
    display(HTML('<div class="plots">'))
    display(HTML('<img align=right src="https://app.aavso.org/vsp/chart/X28194FDL.png" width=380>'))
    display_image(first_image)
    display(HTML('</div>'))

    obs = ""
    display(HTML('<br /><br /><hr /><br /><h3>Data Entry 1 of 2: Enter coordinates for the target star</h3>'))
    display(HTML('<p>Tip: Use the zoom feature. Click the magnifying glass and click-and-drag to draw a rectangle that matches the starchart.</p>'))
    display(HTML('<ol class="step"><li class="step4">In the right image, find the <i>crosshairs</i> in the center - that represents your target star.</li><li class="step4">On the left image, <i>find this target star and roll over it with your mouse</i>, note the X and Y coordinates.</li><li class="step4">Put the X and Y coordinates in the box below in the format <code>[x,y]</code> and press return.</li></ol>')) 
    
    targ_coords = input("Enter coordinates for target star - [424,286] - and press return:  ")  
    # if targ_coords != "[424,286]":
    #   display(HTML('<p class="output">You entered ' + targ_coords + '</p>'))
    targ_coords = "[424,286]"


    showProgress(1)
    display(HTML('<p class="output">Target star coordinates logged</p>'))
    
    display(HTML('<h3>Data Entry 2 of 2: Enter coordinates for at least two comparison stars.</h3>'))
    display(HTML('<ol class="step"><li class="step4">In the right image, find the stars <i>with numbers</i> that represent suggested comparison stars.</li><li class="step4">On the left image, <i>find and roll over each comparison star with your mouse</i> and note the coordinates.</li><li class="step4">Put the X and Y coordinates in the box below in the format <code>[[x1,y1][x2,y2]]</code> and press return.</li></ol>'))
    
    comp_coords = input("Enter coordinates for the comparison stars - [[326,365],[416,343],[491,303]] - and press return:  ")  
    # if comp_coords != "[[326,365],[416,343],[491,303]]":
    #   display(HTML('<p class="step">You entered ' + comp_coords + '<br /><br /></p>'))
    comp_coords = "[[326,365],[416,343],[491,303]]"

    showProgress(1)
    display(HTML('<p class="output">Comparison star coordinates logged</p>'))

    aavso_obs_code = ""

    if not os.path.isdir(output_dir):
      os.mkdir(output_dir)

    inits = [make_inits_file(planetary_params, p, output_dir, first_image, targ_coords, comp_coords, obs, aavso_obs_code, sample_data)]

  if not os.path.isdir(output_dir):    # Make the output directory if it does not exist already.
    os.mkdir(output_dir)
  output_dir_for_shell = output_dir.replace(" ", "\ ")

#print("Path to the inits file(s) that will be used:")

#for inits_file in inits:
#  print(inits_file)

num_inits = len(inits)


display(HTML('<p class="bookend">DONE: Compare telescope image and star chart. <b>You may move on to the next step.</b></p>'))




# **Step 5:  Run EXOTIC**

---
> Estimated time: 5 minutes
---

EXOTIC will analyze the brightness of the target star in each telescop image to identify if there was a transit. At the end of this step, you will see your lightcurve as an image, and the lightcurve data will be downloaded to your hard drive for submission to AAVSO.

In [ ]:
#@title <font face="Helvetica" class="button" color='#702020'>&lt;- Click to run EXOTIC to analyze telescope images (unstyled, simulated)</font>

display(HTML('<p class="bookend">START: Analyzing telescope images</p>'))
setupDisplay()
display(HTML('<ul class="step_container"></ul>'))
log_10 = """
Path to the inits file(s) that will be used:
/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/inits.json
!exotic -red "/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/inits.json" -ov
    Downloading __databases__.pickle_new...
    Done!
Checking exotethys database...
Checking ephemerides database...
Checking photometry database...
Checking catalogues database...

*************************************************************
Welcome to the EXOplanet Transit Interpretation Code (EXOTIC)
Version 1.10.2
*************************************************************


**************************
Complete Reduction Routine
**************************

**************************
Starting Reduction Process
**************************


Getting the plate solution for your imaging file to translate pixel coordinates on the sky. 
Please wait....

"""

log_20 = """
WCS file creation successful.
Thinking ... DONE!

Here is the path to your plate solution: /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/wcs.fits

Checking for variability in Comparison Star #1:
	Pixel X: 493 Pixel Y: 304
 Warning: Your comparison star cannot be resolved in the SIMBAD star database; EXOTIC cannot check if it is variable or not. 
EXOTIC will still include this star in the reduction. 
Please proceed with caution as we cannot check for stellar variability.


Checking for variability in Comparison Star #2:
	Pixel X: 415 Pixel Y: 343
 Warning: Your comparison star cannot be resolved in the SIMBAD star database; EXOTIC cannot check if it is variable or not. 
EXOTIC will still include this star in the reduction. 
Please proceed with caution as we cannot check for stellar variability.


Computing best comparison star, aperture, and sky annulus. Please wait.
"""

log_30 = """

*********************************************
Best Comparison Star: None
Minimum Residual Scatter: 0.672%
Optimal Aperture: 5.02
Optimal Annulus: 7.54
*********************************************

No Barycentric Julian Dates (BJDs) in Image Headers for standardizing time format. Converting all JDs to BJD_TDBs.
Please be patient- this step can take a few minutes.
"""

log_40 = """
Thinking ... DONE!


Output File Saved


****************************************
Fitting a Light Curve Model to Your Data
****************************************

[ultranest] Sampling 400 live points from prior ...


Mono-modal Volume: ~exp(-3.81) * Expected Volume: exp(0.00) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|**********|     +3.0

Z=-1659103.7(0.00%) | Like=-1655784.03..-111.41 [-8987932.8990..-176103.1770] | it/evals=80/494 eff=85.1064% N=400 

Mono-modal Volume: ~exp(-3.96) * Expected Volume: exp(-0.23) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|********  |     +3.0

Z=-338377.3(0.00%) | Like=-334422.26..-111.41 [-8987932.8990..-176103.1770] | it/evals=160/609 eff=76.5550% N=400 

Mono-modal Volume: ~exp(-4.18) * Expected Volume: exp(-0.45) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|*******   |     +3.0

Z=-202573.2(0.00%) | Like=-201882.30..-111.41 [-8987932.8990..-176103.1770] | it/evals=240/730 eff=72.7273% N=400 

"""

log_50 = """
Mono-modal Volume: ~exp(-20.47)   Expected Volume: exp(-17.10) Quality: ok

rprs:    +0.000|        * |   +0.186
tmid: +2458107.706|       ** |+2458107.716
ars :      +4.8|    **    |     +5.9
a2  :    -1.000|    *     |   +1.000

[ultranest] Explored until L=-7e+01  
[ultranest] Likelihood function evaluations: 15883
[ultranest]   logZ = -82.92 +- 0.141
[ultranest] Effective samples strategy satisfied (ESS = 2176.7, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.45+-0.05 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.14, need <0.5)
[ultranest]   logZ error budget: single: 0.16 bs:0.14 tail:0.01 total:0.14 required:<0.50
[ultranest] done iterating.

*********************************************************
FINAL PLANETARY PARAMETERS

          Mid-Transit Time [BJD_TDB]: 2458107.7136 +/- 0.001
  Radius Ratio (Planet/Star) [Rp/Rs]: 0.1587 +/- 0.0037
           Transit depth [(Rp/Rs)^2]: 2.52 +/- 0.12 [%]
 Semi Major Axis/ Star Radius [a/Rs]: 5.36 +/- 0.11
               Airmass coefficient 1: 1.1645 +/- 0.0078
               Airmass coefficient 2: -0.1222 +/- 0.0028
                    Residual scatter: 0.67 %
                 Best Comparison Star: None
                    Optimal Aperture: 5.02
                     Optimal Annulus: 7.54
              Transit Duration [day]: 0.1311 +/- 0.0028
*********************************************************
Output Files Saved

************************
End of Reduction Process
************************
"""

log_60 = """
************************
EXOTIC has successfully run!!!
It is now safe to close this window.
************************
lightcurve: /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/FinalLightCurve_HAT-P-32 b_2017-12-19.png
"""

import time
print(log_10)
showProgress(3)
print(log_20)
showProgress(2)
print(log_30)
showProgress(5)
print(log_40)
showProgress(1)
print(log_50)
showProgress(4)
print(log_60)

display(HTML('<p>The data for the lightcurve you see here is at /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt'))
display(HTML('<p>Downloading now...</p>'))

showProgress(1)

if os.path.isfile('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt'):
  files.download('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt')
else: 
  display(HTML('<p>Couldn\'t find output file. Bergen will work with Rob to ensure it is in there.</p>'))


display(HTML('<p class="bookend">DONE: Analyzing telescope images. <b>You may move on to the next step.</b></p>'))



---

---

# **DONE: Next Steps** 

You have successfully generated a lightcurve showing the transit of Hat-p-32b around Hat-p-32 and downloaded the data to your hard drive as "AAVSO_HAT-P-32 b_2017-12-19.txt", in a format suitable for submission to AAVSO. 

This exoplanet was actually discovered in 2011, and you can [explore it in 3D here](https://exoplanets.nasa.gov/exoplanet-catalog/1434/hat-p-32-b/).

Ready for the next step?  [Use EXOTIC to upload your own data!](https://exoplanets-5.client.mooreboeck.com/exoplanet-watch/exotic/)

---

---

# **APPENDIX: BONUS SEGMENTS**

In [ ]:
#@title <font face="Helvetica" class="button" color='#702020'>&lt;- Click to run EXOTIC to analyze telescope images (developer's note, unstyled, real)</font>

# If the user presses enter to run the sample data, download sample data if needed and
# put it into a sample-data directory at the top level of the user's Gdrive.  Count
# the .fits files (images) and .json files (inits files) in the directory entered 
# by the user (or in the sample-data directory if the user pressed enter).  If 
# there are at least 20 .fits files, assume this is a directory of images and display
# the first one in the series.  If there is exactly one inits file in the directory, 
# show the specified target and comp coords so that the user can check these against
# the displayed image.  Otherwise, prompt for target / comp coords and make an inits 
# file based  on those (save this new inits file in the folder with the output files 
# so that the student can consult it later).  Finally, run EXOTIC with the newly-made 
# or pre-existing inits file, plus any other inits files in the directory.

#########################################################

# p is the name of the folder entered by the user.  Decide what to do based on what
# is found in the folder.
display(HTML('<p class="bookend">START: Analyzing telescope images</p>'))
#display(HTML("<p class='warning'>NOTE: At this point in EXOTIC, you would have the opportunity choose where to temporarily save the sample data. For this exercise, we're downloading to /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017"))
setupDisplay()
display(HTML('<ul class="step_container"></ul>'))
#bokeh.io.output_notebook()
sample_data = False

print("Path to the inits file(s) that will be used:")

for inits_file in inits:
  print(inits_file)

num_inits = len(inits)

commands = []
for inits_file in inits:
  with open(inits_file) as i_file:
    inits_data = i_file.read()
    d = json.loads(inits_data)
    date_obs = d["user_info"]["Observation date"]
    planet = d["planetary_parameters"]["Planet Name"]
    output_dir = d["user_info"]["Directory to Save Plots"]
    if not os.path.isdir(output_dir):
      os.mkdir(output_dir)
    inits_file_for_shell = inits_file.replace(" ", "\\ ")
    run_exotic = str(f"exotic -red {inits_file_for_shell} -ov")
    debug_exotic_run = str(f"!exotic -red \"{inits_file}\" -ov")

    commands.append({"inits_file_for_shell": inits_file_for_shell, "output_dir": output_dir, 
                      "planet": planet, "date_obs": date_obs, 
                      "run_exotic": run_exotic, "debug_exotic_run": debug_exotic_run
                      })
    print(f"{debug_exotic_run}")
    !eval "$run_exotic"

    # Only show lightcurve for beginner - bm
    lightcurve = os.path.join(output_dir,"FinalLightCurve_"+planet+"_"+date_obs+".png")
    fov = os.path.join(output_dir,"temp/FOV_"+planet+"_"+date_obs+"_LinearStretch.png")
    triangle = os.path.join(output_dir,"temp/Triangle_"+planet+"_"+date_obs+".png")
    print(f"lightcurve: {lightcurve}\nfov: {fov}\ntriangle: {triangle}\n")

    if not (os.path.isfile(lightcurve) and os.path.isfile(fov) and os.path.isfile(triangle)):
      print(f"Something went wrong with {planet} {date_obs}.\nCopy the command below into a new cell and run to find the error:\n{debug_exotic_run}\n")
      continue

    imageA = widgets.Image(value=open(lightcurve, 'rb').read())
    imageB = widgets.Image(value=open(fov, 'rb').read())
    hbox = HBox([imageB, imageA])
    # removing for "see it in action" - bm
    display(imageA)
    #display(hbox)
    #display(Image(filename=triangle))

display(HTML('<p>The data for the lightcurve you see here is downloading now in a format suitable to submit to AAVSO.'))

# /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt

showProgress(2)

if os.path.isfile('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt'):
  files.download('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt')
else: 
  display(HTML('<p>Couldn\'t find output file. Bergen will work with Rob to ensure it is in there.</p>'))


display(HTML('<p class="bookend">DONE: Analyzing telescope images. <b>You may move on to the next step.</b></p>'))

    

In [ ]:
#@title <font face="Helvetica" class="button" color='#702020'>&lt;- Click to run EXOTIC to analyze sample data (developer's note: styled, simulated)</font>

display(HTML('<p class="bookend">START: Analyzing telescope images</p>'))
setupDisplay()
display(HTML('<ul class="step_container"></ul>'))

log_10 = """
Path to the inits file(s) that will be used:
/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/inits.json
!exotic -red "/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/inits.json" -ov
    Downloading __databases__.pickle_new...
    Done!
Checking exotethys database...
Checking ephemerides database...
Checking photometry database...
Checking catalogues database...

*************************************************************
Welcome to the EXOplanet Transit Interpretation Code (EXOTIC)
Version 1.10.2
*************************************************************


**************************
Complete Reduction Routine
**************************

**************************
Starting Reduction Process
**************************


Getting the plate solution for your imaging file to translate pixel coordinates on the sky. 
Please wait....

"""

log_20 = """
WCS file creation successful.
Thinking ... DONE!

Here is the path to your plate solution: /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/wcs.fits

Checking for variability in Comparison Star #1:
	Pixel X: 493 Pixel Y: 304
 Warning: Your comparison star cannot be resolved in the SIMBAD star database; EXOTIC cannot check if it is variable or not. 
EXOTIC will still include this star in the reduction. 
Please proceed with caution as we cannot check for stellar variability.


Checking for variability in Comparison Star #2:
	Pixel X: 415 Pixel Y: 343
 Warning: Your comparison star cannot be resolved in the SIMBAD star database; EXOTIC cannot check if it is variable or not. 
EXOTIC will still include this star in the reduction. 
Please proceed with caution as we cannot check for stellar variability.


Computing best comparison star, aperture, and sky annulus. Please wait.
"""

log_30 = """

*********************************************
Best Comparison Star: None
Minimum Residual Scatter: 0.672%
Optimal Aperture: 5.02
Optimal Annulus: 7.54
*********************************************

No Barycentric Julian Dates (BJDs) in Image Headers for standardizing time format. Converting all JDs to BJD_TDBs.
Please be patient- this step can take a few minutes.
"""

log_40 = """
Thinking ... DONE!


Output File Saved


****************************************
Fitting a Light Curve Model to Your Data
****************************************

[ultranest] Sampling 400 live points from prior ...


Mono-modal Volume: ~exp(-3.81) * Expected Volume: exp(0.00) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|**********|     +3.0

Z=-1659103.7(0.00%) | Like=-1655784.03..-111.41 [-8987932.8990..-176103.1770] | it/evals=80/494 eff=85.1064% N=400 

Mono-modal Volume: ~exp(-3.96) * Expected Volume: exp(-0.23) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|********  |     +3.0

Z=-338377.3(0.00%) | Like=-334422.26..-111.41 [-8987932.8990..-176103.1770] | it/evals=160/609 eff=76.5550% N=400 

Mono-modal Volume: ~exp(-4.18) * Expected Volume: exp(-0.45) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|*******   |     +3.0

Z=-202573.2(0.00%) | Like=-201882.30..-111.41 [-8987932.8990..-176103.1770] | it/evals=240/730 eff=72.7273% N=400 

"""

log_50 = """
Mono-modal Volume: ~exp(-20.47)   Expected Volume: exp(-17.10) Quality: ok

rprs:    +0.000|        * |   +0.186
tmid: +2458107.706|       ** |+2458107.716
ars :      +4.8|    **    |     +5.9
a2  :    -1.000|    *     |   +1.000

[ultranest] Explored until L=-7e+01  
[ultranest] Likelihood function evaluations: 15883
[ultranest]   logZ = -82.92 +- 0.141
[ultranest] Effective samples strategy satisfied (ESS = 2176.7, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.45+-0.05 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.14, need <0.5)
[ultranest]   logZ error budget: single: 0.16 bs:0.14 tail:0.01 total:0.14 required:<0.50
[ultranest] done iterating.

*********************************************************
FINAL PLANETARY PARAMETERS

          Mid-Transit Time [BJD_TDB]: 2458107.7136 +/- 0.001
  Radius Ratio (Planet/Star) [Rp/Rs]: 0.1587 +/- 0.0037
           Transit depth [(Rp/Rs)^2]: 2.52 +/- 0.12 [%]
 Semi Major Axis/ Star Radius [a/Rs]: 5.36 +/- 0.11
               Airmass coefficient 1: 1.1645 +/- 0.0078
               Airmass coefficient 2: -0.1222 +/- 0.0028
                    Residual scatter: 0.67 %
                 Best Comparison Star: None
                    Optimal Aperture: 5.02
                     Optimal Annulus: 7.54
              Transit Duration [day]: 0.1311 +/- 0.0028
*********************************************************
Output Files Saved

************************
End of Reduction Process
************************
"""

log_60 = """
************************
EXOTIC has successfully run!!!
It is now safe to close this window.
************************
lightcurve: /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/FinalLightCurve_HAT-P-32 b_2017-12-19.png
"""


displayStep('Using init file at /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/inits.json')

displayStep('Checking exotethys database...')
displayStep('Checking ephemerides database...')
displayStep('Checking photometry database...')
displayStep('Checking catalogues database...')

showProgress(2)
display(HTML('<pre class="log">' + log_10 + ' </pre>'))
showProgress(1)
displayStep('Starting Reduction Process')

displayStep('Getting the plate solution for your imaging file to translate pixel coordinates on the sky.')
showProgress(2)

display(HTML('<pre class="log">' + log_20 + ' </pre>'))

displayStep('WCS file creation successful.')
displayStep('Checking for variability in Comparison Star #1:')
showProgress(1)
displayStep('Checking for variability in Comparison Star #2:')
showProgress(1)

display(HTML('<pre class="log">' + log_30 + ' </pre>'))

displayStep('Computing best comparison star, aperture, and sky annulus. Please wait.')
showProgress(2)

display(HTML('<pre class="log">' + log_40 + ' </pre>'))

displayStep('Converting all JDs to BJD_TDBs.')
showProgress(1)


displayStep('Output File Saved')

display(HTML('<pre class="log">' + log_50 + ' </pre>'))

displayStep('Fitting a Light Curve Model to Your Data')
showProgress(3)

displayStep('Lightcurve Output Files Saved')

display(HTML('<pre class="log">' + log_60 + ' </pre>'))

# print(log_10)
# time.sleep(3)
# print(log_20)
# time.sleep(5)
# print(log_30)
# time.sleep(9)
# print(log_40)
# time.sleep(1)
# print(log_50)
# time.sleep(7)
# print(log_60)

display(HTML('<br /><img src="https://exoplanets.nasa.gov/system/exotic/sample_lightcurve.png" width=500><br />'))


display(HTML('<p>The data for the lightcurve you see here is downloading now in a format suitable to submit to AAVSO.'))

# /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt

showProgress(2)

if os.path.isfile('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt'):
  files.download('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt')
else: 
  display(HTML('<p>Couldn\'t find output file. Bergen will work with Rob to ensure it is in there.</p>'))

display(HTML('<p class="bookend">DONE: Analyzing telescope images. <b>You may move on to the next step.</b></p>'))
